# Content Entry Script

## 1. Script Initiations

In [1]:
import pandas as pd
import numpy as np
import pymssql
from os import getenv
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, URL

In [2]:
# find .env file and load environment variables
load_dotenv(find_dotenv())
server = getenv("SERVER")
username = getenv("USERNAME")
password = getenv("PASSWORD")
db = getenv("DATABASE")

## 2. Data Extractions

In [3]:
# connect with db and query data
url_object = URL.create(
    "mssql+pymssql",
    username=username,
    password=password,
    host=server,
    database=db,
)
engine = create_engine(url_object)
query = """
    SELECT *
    FROM Reports.ContentManagementDetails
    ORDER BY CreatedDate
"""
df = pd.read_sql(query, engine)

## 3. Data Explorations and Processing

In [4]:
# data exploration
print(df.shape)

# find null values
df.isnull().sum()

(10710, 19)


ContentManagementDetailsID       0
ProductID                        0
ProductCategory                  0
ProductSubCategory               0
ProductType                      0
InputCompany                     0
InputCompanyCode                 0
Product                          0
ProductCode                      0
Status                           0
CreatedDate                      0
CreatedBy                        0
UpdatedDate                   8460
UpdatedBy                     8460
IsActive                         0
CreatedBy1                       0
CreatedDate1                     0
ModifiedBy                       7
ModifiedDate                     7
dtype: int64

In [5]:
# drop columns
df.drop(columns=["ContentManagementDetailsID", "ProductID", "CreatedBy1", "ModifiedBy"], inplace=True)

In [6]:
# define datetime columns and convert to date
df["CreatedDate"] = pd.to_datetime(df["CreatedDate"], format="%Y-%m-%d %H:%M:%S.%f").dt.date
df["UpdatedDate"] = pd.to_datetime(df["UpdatedDate"], format="%Y-%m-%d %H:%M:%S.%f").dt.date
df["CreatedDate1"] = pd.to_datetime(df["CreatedDate1"], format="%Y-%m-%d %H:%M:%S.%f").dt.date
df["ModifiedDate"] = pd.to_datetime(df["ModifiedDate"], format="%Y-%m-%d %H:%M:%S.%f").dt.date

# replace None with NaN
df.UpdatedBy.fillna(value=np.nan, inplace=True)

In [7]:
# replace Cultive & PharmapulseUser with NaN
df["CreatedBy"] = np.where(((df["CreatedBy"] == "Cultive") | (df["CreatedBy"] == "PharmapulseUser")), np.nan, df["CreatedBy"])
df["UpdatedBy"] = np.where(((df["UpdatedBy"] == "Cultive") | (df["UpdatedBy"] == "PharmapulseUser")), np.nan, df["UpdatedBy"])

# replace Sanzida with Shama Sanjida
df["CreatedBy"] = np.where((df["CreatedBy"] == "Sanzida"), "Shama Shanjida", df["CreatedBy"])
df["UpdatedBy"] = np.where((df["UpdatedBy"] == "Sanzida"), "Shama Shanjida", df["UpdatedBy"])

# create CreatedByNew column
conditions = [
    (pd.isnull(df["CreatedBy"]) & pd.isnull(df["UpdatedBy"])),
    (pd.isnull(df["CreatedBy"]) & pd.notnull(df["UpdatedBy"])),
    (pd.notnull(df["CreatedBy"]) & pd.isnull(df["UpdatedBy"]))
]
choice = [np.nan, df["UpdatedBy"], df["CreatedBy"]]
df["CreatedByNew"] = np.select(conditions, choice, default=df["CreatedBy"])

## 4. Storing Data

In [8]:
df.to_csv("/mnt/c/Users/Hp/Desktop/query_results/product_entry.csv", index=False)

In [9]:
df.shape

(10710, 16)